In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import numpy as np
np.bool = np.bool_

In [3]:
#current notebook name
notebook_name = __session__.replace('.ipynb','')[__session__.rfind('/')+1:] 

In [4]:
# HDFS base paths
hdfs_lakehouse_base_path = 'hdfs://localhost:9000/lakehouse/'
hdfs_warehouse_base_path = 'hdfs://localhost:9000/warehouse'

In [5]:
import os
dependencies = ["org.apache.spark:spark-avro_2.12:3.5.0",
                "io.delta:delta-iceberg_2.12:3.0.0"]
os.environ['PYSPARK_SUBMIT_ARGS']= f"--packages {','.join(dependencies)} pyspark-shell"
os.environ['PYARROW_IGNORE_TIMEZONE'] = 'true'

In [6]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName(notebook_name)
    .config("spark.log.level","ERROR")
    .config("spark.sql.warehouse.dir",hdfs_warehouse_base_path)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executorEnv.PYTHONPATH", "/path/to/your/venv/lib/python3.11/site-packages") \

    .enableHiveSupport()
    .getOrCreate()
)

25/02/19 13:10:39 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s1)
25/02/19 13:10:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-iceberg_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8d360911-67aa-47e7-a9c7-af07fb29d92a;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-avro_2.12;3.5.0 in central
	found org.tukaani#xz;1.9 in central
	found io.delta#delta-iceberg_2.12;3.0.0 in central
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.1.1 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.3 in central
	found org.checkerframework#checker-qual;3.19.0 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
:: resolution report :: resolve 162ms :: artifacts dl 6ms
	:: modules in use:
	com.github.ben-manes.caffeine#caffeine;2.9.3 from central in [default]
	com.google.errorprone#error_prone_annotations;2.10.0 from central in [default]
	io.delta#delta-iceberg_2.12;3.0.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 f

In [7]:
btc_df = spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/silver/trump_btc/BTC").cache()
trump_df = spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/silver/trump_btc/trump").cache()

In [8]:
btc_df.toPandas()
trump_df.toPandas()


,date,Close,High,Low,Open,Volume
0,2014-09-17,457.3340148925781,468.17401123046875,452.4219970703125,465.864013671875,21056800
1,2014-09-18,424.44000244140625,456.8599853515625,413.10400390625,456.8599853515625,34483200
2,2014-09-19,394.7959899902344,427.8349914550781,384.5320129394531,424.1029968261719,37919700
3,2014-09-20,408.90399169921875,423.2959899902344,389.88299560546875,394.6730041503906,36863600
4,2014-09-21,398.8210144042969,412.4259948730469,393.1809997558594,408.0849914550781,26580100
...,...,...,...,...,...,...
3752,2024-12-25,99299.1953125,99478.75,97593.46875,98675.9140625,33700394629
3753,2024-12-26,95795.515625,99884.5703125,95137.8828125,99297.6953125,47054980873
3754,2024-12-27,94164.859375,97294.84375,93310.7421875,95704.9765625,52419934565
3755,2024-12-28,95163.9296875,95525.8984375,94014.2890625,94160.1875,24107436185


,date,favorites,id,isRetweet,retweets,text
0,2024-10-31 17:16:04,8601.0,1.134029e+17,False,2282.0,"""I would like to wish our great Bitcoiners a Happy 16th Anniversary of Satoshi’s White Paper. We will end Kamala’s war on crypto, &amp; Bitcoin will be MADE IN THE USA! VOTE TRUMP! #Bitcoin #FreeRossDayOne"""
1,2024-09-30 17:11:13,8655.0,1.132271e+17,False,2516.0,"""I promised to Make America Great Again, this time with crypto. #WorldLibertyFi is planning to help make America the crypto capital of the world! The whitelist for eligible persons is officially open – this is your chance to be part of this historic moment. Join: https://www.worldlibertyfinancial.com/"""
2,2024-09-24 17:02:06,0.0,1.131931e+17,True,0.0,"""RT @realtrumpcoinsThe ONLY official coin designed by @realDonaldTrump"
3,2024-09-22 00:22:54,7797.0,1.131779e+17,False,2029.0,"""https://justthenews.com/politics-policy/all-things-trump/trump-uses-bitcoin-buy-hamburgers-new-york"""
4,2024-09-21 23:31:31,7113.0,1.131777e+17,False,1909.0,"""“Trump Credits Success of His NFTs for Change of Heart on Bitcoin and Crypto” https://decrypt.co/249755/trump-credits-nfts-change-of-heart-bitcoin-crypto"""
...,...,...,...,...,...,...
79,2011-10-07 21:21:06,18.0,1.223910e+17,False,58.0,@BarackObama is trying to pass another stimulus after the first was a complete failure. Does he really believe we are that stupid?
80,2011-09-20 15:17:36,11.0,1.161389e+17,False,38.0,"""@BarackObama's method of spending the stimulus -- """"Obama fundraiser linked to loan program that aided Solyndra"""" http://t.co/wSiSn5IP"""
81,2011-09-09 20:30:23,23.0,1.122314e+17,False,80.0,"@BarackObama---the stimulus will fail, just like the first."
82,2011-09-06 22:23:34,105.0,1.111727e+17,False,167.0,@BarackObama trillion $ stimulus was supposed to create jobs -- instead it gave us more debt and higher unemployment.


In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    if text:  
        return analyzer.polarity_scores(text)["compound"]
    return 0.0  

trump_pandas_df=trump_df.toPandas()

trump_pandas_df["sentiment_score"] = trump_pandas_df["text"].apply(analyze_sentiment)

trump_pandas_df = spark.createDataFrame(trump_pandas_df)

# Show results
trump_pandas_df.toPandas()




,date,favorites,id,isRetweet,retweets,text,sentiment_score
0,2024-10-31 17:16:04,8601.0,1.134029e+17,False,2282.0,"""I would like to wish our great Bitcoiners a Happy 16th Anniversary of Satoshi’s White Paper. We will end Kamala’s war on crypto, &amp; Bitcoin will be MADE IN THE USA! VOTE TRUMP! #Bitcoin #FreeRossDayOne""",0.8652
1,2024-09-30 17:11:13,8655.0,1.132271e+17,False,2516.0,"""I promised to Make America Great Again, this time with crypto. #WorldLibertyFi is planning to help make America the crypto capital of the world! The whitelist for eligible persons is officially open – this is your chance to be part of this historic moment. Join: https://www.worldlibertyfinancial.com/""",0.9151
2,2024-09-24 17:02:06,0.0,1.131931e+17,True,0.0,"""RT @realtrumpcoinsThe ONLY official coin designed by @realDonaldTrump",0.0000
3,2024-09-22 00:22:54,7797.0,1.131779e+17,False,2029.0,"""https://justthenews.com/politics-policy/all-things-trump/trump-uses-bitcoin-buy-hamburgers-new-york""",0.0000
4,2024-09-21 23:31:31,7113.0,1.131777e+17,False,1909.0,"""“Trump Credits Success of His NFTs for Change of Heart on Bitcoin and Crypto” https://decrypt.co/249755/trump-credits-nfts-change-of-heart-bitcoin-crypto""",0.8860
...,...,...,...,...,...,...,...
79,2011-10-07 21:21:06,18.0,1.223910e+17,False,58.0,@BarackObama is trying to pass another stimulus after the first was a complete failure. Does he really believe we are that stupid?,-0.7717
80,2011-09-20 15:17:36,11.0,1.161389e+17,False,38.0,"""@BarackObama's method of spending the stimulus -- """"Obama fundraiser linked to loan program that aided Solyndra"""" http://t.co/wSiSn5IP""",0.0000
81,2011-09-09 20:30:23,23.0,1.122314e+17,False,80.0,"@BarackObama---the stimulus will fail, just like the first.",-0.2500
82,2011-09-06 22:23:34,105.0,1.111727e+17,False,167.0,@BarackObama trillion $ stimulus was supposed to create jobs -- instead it gave us more debt and higher unemployment.,-0.5563
